In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('judge-1377884607_tweet_product_company.csv', encoding = 'unicode_escape')
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 3 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     3291 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9093 non-null   object
dtypes: object(3)
memory usage: 213.2+ KB


In [6]:
df['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts()

No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

In [7]:
df['emotion_in_tweet_is_directed_at'].value_counts()

iPad                               946
Apple                              661
iPad or iPhone App                 470
Google                             430
iPhone                             297
Other Google product or service    293
Android App                         81
Android                             78
Other Apple product or service      35
Name: emotion_in_tweet_is_directed_at, dtype: int64

In [12]:
binary_df = df[(df['is_there_an_emotion_directed_at_a_brand_or_product'] == 'Positive emotion') | 
               (df['is_there_an_emotion_directed_at_a_brand_or_product'] == 'Negative emotion')]

binary_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3548 entries, 0 to 9088
Data columns (total 3 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          3548 non-null   object
 1   emotion_in_tweet_is_directed_at                     3191 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  3548 non-null   object
dtypes: object(3)
memory usage: 110.9+ KB


In [14]:
binary_df_nulls = binary_df[binary_df['emotion_in_tweet_is_directed_at'].isna() == True]
binary_df_nulls

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
46,Hand-Held Û÷HoboÛª: Drafthouse launches Û÷H...,NaN,Positive emotion
64,Again? RT @mention Line at the Apple store is ...,NaN,Negative emotion
68,Boooo! RT @mention Flipboard is developing an ...,NaN,Negative emotion
103,Know that &quot;dataviz&quot; translates to &q...,NaN,Negative emotion
112,Spark for #android is up for a #teamandroid aw...,NaN,Positive emotion
...,...,...,...
9011,apparently the line to get an iPad at the #sxs...,NaN,Positive emotion
9043,Hey is anyone doing #sxsw signing up for the g...,NaN,Negative emotion
9049,@mention you can buy my used iPad and I'll pic...,NaN,Positive emotion
9052,@mention You could buy a new iPad 2 tmrw at th...,NaN,Positive emotion


In [39]:
binary_df_nonulls = binary_df.dropna()
binary_df_nonulls = binary_df_nonulls.drop(columns = 'emotion_in_tweet_is_directed_at')
binary_df_nonulls['is_there_an_emotion_directed_at_a_brand_or_product'] = binary_df_nonulls['is_there_an_emotion_directed_at_a_brand_or_product'].apply(lambda x: 0 if x == 'Negative emotion' else 1)
binary_df_nonulls

,tweet_text,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,0
1,@jessedee Know about @fludapp ? Awesome iPad/i...,1
2,@swonderlin Can not wait for #iPad 2 also. The...,1
3,@sxsw I hope this year's festival isn't as cra...,0
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,1
...,...,...
9077,@mention your PR guy just convinced me to swit...,1
9079,&quot;papyrus...sort of like the ipad&quot; - ...,1
9080,Diller says Google TV &quot;might be run over ...,0
9085,I've always used Camera+ for my iPhone b/c it ...,1


NLTK (Natural Language ToolKit)
- Stop Word Removal - reduce dimensionality, frequency distributions

from nltk.corpus import stopwords
import string

stopwords_list = stopwords.words('english')

stopwords_list += list(string.punctuation)

from nltk import word_tokenize

tokens = word_tokenize(some_text_data)

stopped_tokens = [w.lower() for w in tokens if w not in stopwords_list]

from  nltk import FreqDist
freqdist = FreqDist(tokens)

most_common = freqdist.most_common(200)

- Regular Expressions - Useful during Tokenization (define rules for splitting strings into separate tokens)


- Stemming and Lemmatization

from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

lemmatizer.lemmatize('feet') # foot
lemmatizer.lemmatize('running') # run

- Bigrams and Mutual Information Score

In [16]:
import nltk
nltk.download('stopwords', quiet=True)
from nltk.corpus import stopwords
from nltk.collocations import *
from nltk import FreqDist
from nltk import word_tokenize
import string
import re

## Pre-Processing
- Capitalization
- Apostrophes
- Numbers

## Frequency Distributions
- Freq_Dist()

## Removing Stop Words and Punctuation

stopwords_list = stopwords.words('english')
stopwords_list += list(string.punctuation)
stopwords_list += ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

macbeth_words_stopped = [word for word in macbeth_tokens if word not in stopwords_list]

macbeth_stopped_freqdist = FreqDist(macbeth_words_stopped)
macbeth_stopped_freqdist.most_common(50)

## Answering Questions about the Text
- Vocabulary Size
- Normalized Word Frequency

total_word_count = sum(macbeth_stopped_freqdist.values())
macbeth_top_50 = macbeth_stopped_freqdist.most_common(50)
print(f'{"Word":10} Normalized Frequency')
for word in macbeth_top_50:
    normalized_frequency = word[1] / total_word_count
    print(f'{word[0]:10} {normalized_frequency:^20.4}')

## Creating Bigrams

We'll begin by aliasing a particularly long method name to make it easier to call. Store nltk.collocations.BigramAssocMeasures() inside of the variable bigram_measures
Next, we'll need to create a finder. Pass macbeth_words_stopped into BigramCollocationFinder.from_words() and assign the result to macbeth_finder
Once we have a finder, we can use it to compute bigram scores, so we can see the combinations that occur most frequently. Call the macbeth_finder object's score_ngrams() method and pass in bigram_measures.raw_freq as the input
Display first 50 elements in the macbeth_scored list to see the 50 most common bigrams in macbeth

## Using Mutual Information Scores

We'll start by creating another finder for pmi. Pass macbeth_words_stopped as the input to BigramCollocationFinder.from_words(). Store this is the variable macbeth_pmi_finder
Once we have our finder, we'll need to apply our frequency filter. Call macbeth_pmi_finder's apply_freq_filter and pass in the number 5 as the input
Now, we can use the finder to calculate pmi scores. Use the pmi finder's .score_ngrams() method, and pass in bigram_measures.pmi as the argument. Store this in macbeth_pmi_scored
Examine the first 50 elements in macbeth_pmi_scored